In [55]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

base_path = "/Users/saraawad/Desktop/Datasets/Google/"
hourly_classified_path = os.path.join(base_path + "Ameriflux/", "Ameriflux Hourly Classified/")
daily_classified_path = os.path.join(base_path + "Ameriflux/", "Jan-11-2020_Hourly/")

<h3> Helpers </h3>

In [4]:
class Helpers:
    def __init__(self):
        print("Helper")
        
    def convert_missing_values_nan(df):
        '''This function will convert -9999 to NaN'''
        df = df.replace(-9999.000000, np.NaN)
        return df

    def drop_nan_columns(df):
        '''Drops the columns having all theirs rows as Nans'''
        columns_to_exclude = ["Date", "Day", "Year", "Month", "Timestamp start"
                              , "Time", "TIMESTAMP", "Tier", "TIMESTAMP_START", "TIMESTAMP_END", "Day Status"]
        columns = df.columns
        for i in range(len(columns)):
            col = columns[i]
            if col in columns_to_exclude:
                continue
            nan_sum_col = df[col].isnull().sum()
            if nan_sum_col == len(df):
                df.drop(col, axis=1, inplace=True)
        return df
    
    def drop_nans_rows(df):
        '''This function will drop the rows having NaNs'''
        print("Before removing missing values:")
        print("number of rows:", df.shape[0], "\nnumber of columns:", df.shape[1])
        df = df.dropna(how='any')
        print("After removing missing values:")
        print("number of rows:", df.shape[0], "\nnumber of columns:", df.shape[1])
        return df
        
    def get_all_matching_columns(df, keyword):
        return df.filter(like=keyword).columns

    def generate_lags(df, column, lags_count): 
        for i in range(lags_count):
            lag_name = column + "-" + str(i + 1)
            df[lag_name] = df[column].shift(i + 1)
#             for j in range(i):
#                 df.loc[str(j+1), lag_name] = np.nan
#         df = df.dropna(how='any')
        return df

    def add_LE_conversion_rate(df, col):
        conversion_rate = 28.94
        new_col = col + "(mm)"
        df[new_col] = df[col] / conversion_rate
        return df

    def read_sites_data():
        file_path = os.path.join(base_path, "filtered_sites_all.xlsx")
        df = pd.read_excel(file_path)
        df.head()
        return df

    def export_data(df, file_path):
        export_path = os.path.join(base_path, file_path + ".csv")
        export_csv = df.to_csv(export_path, index=None, header=True)

    def load_data(file_path):
        df = pd.read_csv(file_path + ".csv", delimiter=',')
        return df
    
    def list_to_df(list_to_convert):
        '''This function will convert the provided list into a dataframe'''
        df = pd.concat(list_to_convert, sort=True)
        return df
    
    def get_files_directory(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
        listOfFile = os.listdir(dirName)
        allFiles = list()
        # Iterate over all the entries
        for entry in listOfFile:
            # Create full path
            if entry.endswith(".xlsx") or entry.endswith(".icloud") or entry.endswith(".DS_Store"):
                continue
            fullPath = os.path.join(dirName, entry)
            # If entry is a directory then get the list of files in this directory 
            if os.path.isdir(fullPath):
                allFiles = allFiles + Helpers.get_files_directory(fullPath)
            else:
                allFiles.append(fullPath)

        return allFiles

    def concat_dataframe_from_files(files, skipRowsNum, split_num):
        values = []
        for i in range(len(files)):
            file_path = files[i]
            head, file_name = os.path.split(file_path)
            #Get only the sheets having the variables
            if file_name.endswith(".csv"):
#                 print("file name", file_name)
                df = pd.read_csv(file_path, delimiter=',', skiprows=skipRowsNum)
                site_id = file_name.split("_")[split_num]
#                 print("site id in file:", site_id)
                df["Site Id"] = site_id
                values.append(df)
        return Helpers.list_to_df(values)   
    
    def generate_dataframe_from_files(dirName, skipRowsNum = 0, split_num = 0):
        files = Helpers.get_files_directory(dirName)
        df = Helpers.concat_dataframe_from_files(files, skipRowsNum, split_num)
        return df
        

<h3> Ameriflux & Joint Class </h3>

In [5]:
class Ameriflux:

    def __init__(self, folder_path, skipRowsNum, split_num, lags_count, is_hourly, is_joint, output_name):
        print("Initializer")
        self.folder_path = folder_path
        self.skipRowsNum = skipRowsNum
        self.split_num = split_num
        self.lags_count = lags_count
        self.is_hourly = is_hourly
        self.is_joint = is_joint
        self.output_name = output_name
        
        
    def are_bowes_elements_exists(self, df):
        bowens_columns = ["H", "LE", "NETRAD", "G"]
        if all([item in df.columns for item in bowens_columns]):
            return True
        else:
            return False
        
    def compute_bowens_elements(self, df):
        if self.are_bowes_elements_exists(df):
            df["LATENT_SENSIBLE"] = np.where(df["NETRAD"] > 20, df["H"] + df["LE"], 0)
            df["NETRAD_SOIL"] = np.where(df["NETRAD"] > 20, df["NETRAD"] + df["G"], 0)
        return df
    
    def compute_bowens_ratio(self, df):
        bowens_columns = ["LATENT_SENSIBLE", "NETRAD_SOIL"]
        if all([item in df.columns for item in bowens_columns]):
            df["C_BOWENS"] = np.divide(df["LATENT_SENSIBLE"], df["NETRAD_SOIL"])
            #If net radiation is not valid set the corrected columns as NaN and then remove their rows
            df["LE_CORRECTED"] = np.where(df["C_BOWENS"] > 0, np.divide(df["LE"], df["C_BOWENS"]), np.NaN)
            df["H_CORRECTED"] = np.where(df["C_BOWENS"] > 0, np.divide(df["H"], df["C_BOWENS"]), np.NaN)  
            df.dropna(subset=["LE_CORRECTED", "H_CORRECTED"], how='all', inplace=True)
            df.drop(bowens_columns, axis=1, inplace=True)
        return df
            
    def impute_temperature(self, df):
        '''This function imputes the temperature by the mean when TA is negative otherwise set it to 0 
        if the mean is negative'''
        columns_list = ["TA"]

        #Get the mean air temperature, if less than zero
        #fall back to zero and then delete the mean column
        for i in range(len(columns_list)):
            col = columns_list[i]
            new_col = col + "-avg"
            df[new_col] = df[col].mean()
            df[new_col] = np.where(df[new_col] < 0, 0, df[new_col])
            df[col] = np.where(df[col] < 0, df[new_col], df[col])
            
        #Drop the new mean columns that are generated temporarly
        new_columns_lists = []
        for i in range(len(columns_list)):
            col = columns_list[i]
            new_col = col + "-avg" 
            new_columns_lists.append(new_col)
        df.drop(new_columns_lists, axis=1, inplace=True)
        
        return df
        
    def transform_input_variants(self, df):
        '''This function gets all the input column variants'''
        ws_list = list(Helpers.get_all_matching_columns(df, "WS_"))
        rh_list = list(Helpers.get_all_matching_columns(df, "RH_"))
        ta_list = list(Helpers.get_all_matching_columns(df, "TA_"))
        g_list = list(Helpers.get_all_matching_columns(df, "G_"))
        h_list = [col for col in df if col.startswith('H_')]
        netrad_list = list(Helpers.get_all_matching_columns(df, "NETRAD_"))
        
        df = self.group_input_variants(df, ws_list, "WS")
        df = self.group_input_variants(df, rh_list, "RH")
        df = self.group_input_variants(df, ta_list, "TA")
        df = self.group_input_variants(df, g_list, "G")
        df = self.group_input_variants(df, h_list, "H")
        df = self.group_input_variants(df, netrad_list, "NETRAD")
        df = self.impute_temperature(df)
        print("After grouping", df.columns)
        return df
        
    def group_input_variants(self, df, variant_list, mean_column):
        '''This function imputes all the input columnn variants with the mean of them and drop the variants'''
        if len(variant_list) > 1:
            df[mean_column] = ""
            df[mean_column] = df[variant_list].mean(axis=1)
        elif len(variant_list) > 0:
             df[mean_column] = df[variant_list[0]]
        
        df = df.drop(variant_list, axis=1)
        return df
    
    def check_If_TA_has_negative(self, df):
        '''This function counts the number of negative air temperature'''
        if "TA" in df.columns:
            print("TA negative values count:", df["TA"].lt(0).sum())
            
    def generate_input_lags(self, df):
        '''This function generates the lags for the list of input columns'''
        input_columns = ["RH", "TA", "G", "WS", "NETRAD"]
        h_corr_name = "H_CORRECTED"
        if (h_corr_name in df.columns):
            input_columns.append(h_corr_name)
        elif "H" in df.columns:
            input_columns.append("H")
        
        for k in range(len(input_columns)):
            col = input_columns[k]
            if col in df.columns:
                df = Helpers.generate_lags(df, col, self.lags_count)
        return df
    
    def add_LE_converstion_to_lags(self, df):
        '''This function adds the conversion for LE incase LE exists and generate lags 
        for it after adding the conversion'''
        #Remove LE for sites having a variant of LE column
        le_corrected_name = "LE_CORRECTED"
        columns_to_drop = (list(df.filter(like='LE_').columns))
        if le_corrected_name in columns_to_drop:
            columns_to_drop.remove(le_corrected_name)
        df.drop(columns_to_drop, axis=1, inplace=True)
        
        if le_corrected_name in df.columns:
            column_to_convert = le_corrected_name
        elif "LE" in df.columns:
            column_to_convert = "LE"
        if column_to_convert != "":
            df = Helpers.add_LE_conversion_rate(df, column_to_convert)
            columns_list = (list(df.filter(like='LE').columns))
            filt_col_list = [col for col in columns_list if "(mm)" in col]
            for k in range(len(filt_col_list)):
                col = filt_col_list[k]
                if col in df.columns:
                    df = Helpers.generate_lags(df, col, self.lags_count)   
        return df
        
    def remove_unneeded_columns_hourly(self, df):
        main_columns = self.order_hourly_columns(df)
        main_columns.extend(["WS", "RH", "TA", "G",  "H", "NETRAD", "LE"])
        main_columns_exists = False
        if all([item in df.columns for item in main_columns]):
            #Remove rows having NaNs
            df = df[main_columns]
            df = Helpers.drop_nans_rows(df)
            df = self.generate_input_lags(df)
            df = self.add_LE_converstion_to_lags(df)
            df = self.order_columns(df)
            main_columns_exists = True
        return (df, main_columns_exists)
    
    def remove_unneeded_columns_daily(self, df):
        main_columns = self.order_daily_columns(df)
        main_columns.extend(["WS", "RH", "TA", "G",  "H", "NETRAD", "LE", "LATENT_SENSIBLE", "NETRAD_SOIL"])
        main_columns_exists = False
        if all([item in df.columns for item in main_columns]):
            #Remove rows having NaNs
            df = df[main_columns]
            df = Helpers.drop_nans_rows(df)
            df = self.compute_bowens_ratio(df)
            df = self.generate_input_lags(df)
            df = self.add_LE_converstion_to_lags(df)
            df = self.order_columns(df)
            main_columns_exists = True
        return (df, main_columns_exists)
    
            
    def order_columns(self, df):
        '''This function will specify the columns required and will order the columns'''
        all_columns = self.order_hourly_columns(df) if is_hourly else self.order_daily_columns(df)
        if "WS" in df.columns:
            all_columns.append("WS")
        ws_list = [col for col in df if col.startswith('WS-')]
        all_columns.extend(ws_list)
        if "RH" in df.columns:
            all_columns.append("RH")
        rh_list = [col for col in df if col.startswith('RH-')]
        all_columns.extend(rh_list)
        if "TA" in df.columns:
            all_columns.append("TA")
        ta_list = [col for col in df if col.startswith('TA-')]
        all_columns.extend(ta_list)
        if "G" in df.columns:
            all_columns.append("G")
        g_list = [col for col in df if col.startswith('G-')]
        all_columns.extend(g_list)
        if "H" in df.columns:
            all_columns.append("H")
            h_list = [col for col in df if col.startswith('H-')]
            all_columns.extend(h_list)
        if "H_CORRECTED" in df.columns:
            all_columns.append("H_CORRECTED")
            h_list = [col for col in df if col.startswith('H_CORRECTED-')]
            all_columns.extend(h_list)
            
        netrad_list = [col for col in df if col.startswith('NETRAD')]
        all_columns.extend(netrad_list)
        
        if "LE_CORRECTED" not in df.columns:
            le_list = [col for col in df if col.startswith('LE')]
            all_columns.extend(le_list)
        else:
            all_columns.append("LE")
            le_list = [col for col in df if col.startswith('LE_CORRECTED')]
            all_columns.extend(le_list)
 
        bowens_columns = ["LATENT_SENSIBLE", "NETRAD_SOIL"]
        if (all([item in df.columns for item in bowens_columns])) and (self.is_hourly):
            all_columns.extend(bowens_columns)
            
        if "C_BOWENS" in df.columns:
            all_columns.append("C_BOWENS")
        print("ordered columns", all_columns)
        df = df[all_columns]
        return df
    
    def order_hourly_columns(self, df):
        '''This function will specify the calendar and main columns for the hourly data'''
        all_columns = ["Timestamp start", "TIMESTAMP_END", "Site Id", "Year", "Month", "Day"]
        return all_columns

    def order_daily_columns(self, df):
        '''This function will specify the calendar and main columns for the daily data'''
        all_columns = ["Date", "Site Id", "Year", "Month", "Day"]
        return all_columns
    
    def resample_date_todays(self, df):
        '''This function will add a date column removing the hours and minutes'''
        time_stamp = []
        index_date_column = 0
        if "TIMESTAMP_END" in df.columns:
            index_date_column = df.columns.get_loc("TIMESTAMP_END")
        elif "TIMESTAMP_start" in df.columns:
            index_date_column = df.columns.get_loc("TIMESTAMP_start")
        for i in range(df.shape[0]):
            date = datetime.strptime(str(df.iloc[i, index_date_column]), "%Y%m%d%H%M").strftime('%m/%d/%y')
            time_stamp.append(date)
        df["Date"] = time_stamp
        return df

    def generate_date_components(self, df):
        '''This function creates new columns for year, month, day from the time stamp
        and converts the data frame into a time series with the data sorted
        in an ascending order'''
        years = []
        months = []
        days = []
        full_dates = []
        for j in range(df.shape[0]):
            #Gets the index of time stamp start column
            index_date_column = df.columns.get_loc("Date")
            full_date_time = str(df.iloc[j, index_date_column])
            years.append(full_date_time[0:4])
            months.append(full_date_time[5:7])
            days.append(full_date_time[8:10])
            full_dates.append(full_date_time[0:10])
        df["Year"] = years
        df["Month"] = months
        df["Day"] = days
        df["Date"] = full_dates
        df["Full Date"] = full_dates
        df["Full Date"] = pd.to_datetime(df["Full Date"])
        df = df.set_index('Full Date')
        df.sort_values(by=["Full Date"], inplace=True)
        return df

    def group_dates_mean(self, df):
        '''This function will resample the half-hourly data to daily'''
        avg_columns = []
        sum_columns = []
        if "RH" in df.columns:
            avg_columns.append("RH")
        if "NETRAD" in df.columns:
            avg_columns.append("NETRAD")
        if "TA" in df.columns:
            avg_columns.append("TA")
        if "WS" in df.columns:
            avg_columns.append("WS")
        if "H" in df.columns:
            avg_columns.append("H")
        if "G" in df.columns:
            avg_columns.append("G")
        if "LE" in df.columns:
            avg_columns.append("LE")
            
        bowens_columns = ["LATENT_SENSIBLE", "NETRAD_SOIL"]
        if all([item in df.columns for item in bowens_columns]):
            sum_columns.extend(bowens_columns)
        
        dictionary_avg = {
        i : np.mean for i in avg_columns
        }
        dictionary_sum = {
        i : sum for i in sum_columns
        }
        result_dictionary = {**dictionary_avg , **dictionary_sum}
        """Get the average of the variables for each day"""
        df = df.groupby(['Site Id','Date'], as_index=False).agg(result_dictionary)
        df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%y")
        df = df.sort_values(by='Date')
        return df
    
    def drop_invalid_columns(self, df):
        '''This function will remove un-needed columns that have different unit of measure than the
        other variants so they should be dropped before grouping variants'''
        ssitc_list = list(Helpers.get_all_matching_columns(df, "_SSITC_TEST"))
        max_list = list(Helpers.get_all_matching_columns(df, "WS_MAX"))
        columnsToDrop = []
        columnsToDrop.extend(ssitc_list)
        columnsToDrop.extend(max_list)
        print("columns to drop", columnsToDrop)
        df = df.drop(columnsToDrop, axis=1)
        return df
    
    def generate_hourly_data(self, df):
        '''This function will process the half-hourly data'''
        df = self.drop_invalid_columns(df)
        #-9999 is converted to NaNs
        df = Helpers.convert_missing_values_nan(df)
        #Remove rows having NaNs
        df = Helpers.drop_nans_rows(df)
        print("df shape:", df.shape)
        df = self.transform_input_variants(df)
        (df, status) = self.remove_unneeded_columns_hourly(df)
        df = self.compute_bowens_elements(df)
        return (df, status)
    
    def generate_daily_data(self, df):
        '''This function will process the daily data'''
        df = self.resample_date_todays(df)
        df = self.group_dates_mean(df)
        df = self.generate_date_components(df)
        (df, status) = self.remove_unneeded_columns_daily(df)
        return (df, status)
        
    def generate_site_data(self, sites_df):
        files = Helpers.get_files_directory(self.folder_path)
        sites = []
        group_sites_list = []
        sites_dict = {}
        for i in range(len(files)):
            file_path = files[i]
            head, file_name = os.path.split(file_path)
            #Get only the sheets having the variables
            if file_name.endswith(".csv"):
                df_filt = pd.read_csv(file_path, delimiter=',', skiprows=self.skipRowsNum)
                site_id = file_name.split("_")[self.split_num]
                df_filt["Site Id"] = site_id
                print("Site:", site_id)
                if self.is_hourly:
                    (df_filt, status) = self.generate_hourly_data(df_filt)
                    if not status:
                        continue
                else:
                    (df_filt, status) = self.generate_daily_data(df_filt)
                    if not status:
                        continue
                    print("columns:", df_filt.columns)
                    
                #Concat all hours updated to a list
                if (len(df_filt) > 0) and ("LE" in df_filt.columns) :
                    sites.append(site_id)
                    group_sites_list.append(df_filt)
                    self.check_If_TA_has_negative(df_filt)
#                     df_final = df_filt.replace(np.NaN, 0)
                    sites_dict[site_id] = df_filt.columns 
                    
                    
                    suffix_name = "Hourly" if is_hourly else "Daily"
                    file_name = os.path.join(self.output_name, site_id + "_" + suffix_name)
                    Helpers.export_data(df_filt, file_name) 

            sites_dict_df = pd.DataFrame.from_dict(sites_dict, orient='index').transpose()
            file_name = os.path.join(self.output_name, "Sites_Variables")
            Helpers.export_data(sites_dict_df, file_name)  
        
        print("Valid Sites are", sites)
        return Helpers.list_to_df(group_sites_list)   

<h3> Ameriflux Hourly Data </h3>

In [6]:
if __name__ == "__main__":
    is_hourly = True #Boolean to indicate if the data is hourly or daily
    skipRowsNum = 0 #Defaults to zero, incase excel has meaningless rows to skip
    split_num = 0 #The index to read the name of the site, defaults to 0
    lags_count = 5 #The number of lags to generate the data for 
    output_name = os.path.join(base_path + "Ameriflux/", "Jan-11-2020_Hourly/")
    am = Ameriflux(hourly_classified_path, skipRowsNum, split_num, lags_count, is_hourly, False, output_name)
    sites_df = Helpers.read_sites_data()
    all_sites_df = am.generate_site_data(sites_df)
    all_sites_df = am.order_columns(all_sites_df)
    file_name = os.path.join(output_name, "All_Hourly")
    Helpers.export_data(all_sites_df, file_name) 
    print(all_sites_df.head())

Initializer
Site: US-Twt
columns to drop ['FC_SSITC_TEST', 'FCH4_SSITC_TEST', 'H_SSITC_TEST', 'LE_SSITC_TEST', 'TAU_SSITC_TEST_PI_F']
Before removing missing values:
number of rows: 144768 
number of columns: 47
After removing missing values:
number of rows: 50155 
number of columns: 47
df shape: (50155, 47)
After grouping Index(['TIMESTAMP_END', 'CO2', 'H2O', 'CH4', 'FC', 'FCH4', 'G', 'H', 'LE',
       'WD', 'WS', 'USTAR', 'ZL', 'TAU', 'MO_LENGTH', 'V_SIGMA', 'W_SIGMA',
       'PA', 'TA', 'VPD_PI', 'T_SONIC', 'T_SONIC_SIGMA', 'TS_PI_1', 'TS_PI_2',
       'TS_PI_3', 'TS_PI_4', 'TS_PI_5', 'WTD', 'NETRAD', 'PPFD_IN', 'PPFD_OUT',
       'SW_IN', 'P', 'FC_PI_F', 'RECO_PI_F', 'GPP_PI_F', 'FCH4_PI_F',
       'LE_PI_F', 'Site Id', 'Category', 'Year', 'Month', 'Day', 'Date',
       'Timestamp start', 'RH'],
      dtype='object')
Before removing missing values:
number of rows: 50155 
number of columns: 13
After removing missing values:
number of rows: 50155 
number of columns: 13
ordered column

Site: US-Snd
columns to drop []
Before removing missing values:
number of rows: 140256 
number of columns: 35
After removing missing values:
number of rows: 44511 
number of columns: 35
df shape: (44511, 35)
After grouping Index(['TIMESTAMP_END', 'USTAR', 'TA', 'WD', 'WS', 'NEE_PI', 'FC', 'SC', 'H',
       'LE', 'G', 'TS_1', 'TS_2', 'P', 'RH', 'PA', 'CO2_1', 'CO2_2', 'VPD_PI',
       'SWC_1', 'SWC_2', 'NETRAD', 'PPFD_IN', 'SW_IN', 'PPFD_OUT', 'H2O',
       'PPFD_DIF', 'ZL', 'Site Id', 'Category', 'Year', 'Month', 'Day', 'Date',
       'Timestamp start'],
      dtype='object')
Before removing missing values:
number of rows: 44511 
number of columns: 13
After removing missing values:
number of rows: 44511 
number of columns: 13
ordered columns ['Timestamp start', 'TIMESTAMP_END', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5'

Site: US-Var
columns to drop []
Before removing missing values:
number of rows: 342528 
number of columns: 67
After removing missing values:
number of rows: 50215 
number of columns: 67
df shape: (50215, 67)
After grouping Index(['TIMESTAMP_END', 'FC', 'CO2', 'CO2_SIGMA', 'LE', 'H2O', 'H2O_SIGMA',
       'H', 'T_SONIC', 'T_SONIC_SIGMA', 'WD', 'WS', 'USTAR', 'W_SIGMA',
       'U_SIGMA', 'V_SIGMA', 'TA', 'VPD_PI', 'RH', 'PA', 'TS_PI_1_1_A',
       'TS_PI_1_2_A', 'TS_PI_1_3_A', 'TS_PI_1_4_A', 'TS_PI_1_5_A',
       'SWC_PI_1_1_A', 'SWC_PI_1_2_A', 'SWC_PI_1_3_A', 'P', 'ZL', 'NETRAD',
       'SW_IN_1_1_1', 'PPFD_IN', 'PPFD_OUT', 'SW_IN_1_1_2', 'SW_OUT', 'LW_IN',
       'LW_OUT', 'PPFD_DIR', 'PPFD_DIF', 'NEE_PI_F', 'GPP_PI_F', 'RECO_PI_F',
       'FC_PI_F', 'LE_PI_F', 'VPD_PI_F', 'PA_PI_F', 'TS_PI_F_1_1_A',
       'TS_PI_F_1_2_A', 'TS_PI_F_1_3_A', 'TS_PI_F_1_4_A', 'TS_PI_F_1_5_A',
       'SWC_PI_F_1_1_A', 'SWC_PI_F_1_2_A', 'SWC_PI_F_1_3_A', 'P_PI_F',
       'Site Id', 'Category', 'Year', 'Mon

Before removing missing values:
number of rows: 154847 
number of columns: 72
After removing missing values:
number of rows: 0 
number of columns: 72
df shape: (0, 72)
After grouping Index(['TIMESTAMP_END', 'FC', 'LE', 'H', 'CO2', 'H2O', 'PA', 'T_SONIC',
       'T_SONIC_SIGMA', 'RH', 'P', 'PPFD_IN', 'SW_IN', 'SW_OUT', 'LW_IN',
       'LW_OUT', 'NETRAD', 'ALB', 'SWC_1_1_1', 'SWC_1_2_1', 'SWC_1_3_1',
       'SWC_1_4_1', 'SWC_1_5_1', 'SWC_2_1_2', 'SWC_2_2_2', 'SWC_2_3_2',
       'SWC_3_4_2', 'SWC_2_5_2', 'SWC_3_1_3', 'SWC_3_2_3', 'SWC_3_3_3',
       'SWC_3_4_3', 'SWC_3_5_3', 'SWC_4_1_1', 'SWC_5_1_1', 'SWC_6_1_1',
       'TS_1_1_1', 'TS_1_2_1', 'TS_1_3_1', 'TS_1_4_1', 'TS_1_5_1', 'TS_1_6_1',
       'D_SNOW', 'USTAR', 'U_SIGMA', 'V_SIGMA', 'W_SIGMA', 'WS', 'WD',
       'SWC_1_1_2', 'SWC_1_2_2', 'SWC_1_3_2', 'SWC_1_4_2', 'SWC_1_5_2',
       'SWC_1_1_3', 'SWC_1_2_3', 'SWC_1_3_3', 'SWC_1_4_3', 'SWC_1_5_3',
       'Site Id', 'Category', 'Year', 'Month', 'Day', 'Date',
       'Timestamp start', 

df shape: (41783, 33)
After grouping Index(['TIMESTAMP_END', 'USTAR', 'WD', 'WS', 'FC', 'H', 'LE', 'G', 'TS_1_1_1',
       'TS_1_2_1', 'P', 'PA_1_1_1', 'CO2', 'VPD_PI_1_1_1', 'SWC_1_1_1',
       'SWC_1_2_1', 'NETRAD', 'PPFD_IN', 'SW_IN', 'PPFD_OUT', 'SW_OUT',
       'LW_IN', 'LW_OUT', 'H2O', 'Site Id', 'Category', 'Year', 'Month', 'Day',
       'Date', 'Timestamp start', 'RH', 'TA'],
      dtype='object')
Before removing missing values:
number of rows: 41783 
number of columns: 13
After removing missing values:
number of rows: 41783 
number of columns: 13
ordered columns ['Timestamp start', 'TIMESTAMP_END', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H-1', 'H-2', 'H-3', 'H-4', 'H-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE(mm)', 'LE(mm)-1', 'LE(mm)-2', 'LE(mm)-3', 'LE(mm)-4'

<h3> Ameriflux Daily Data </h3>

In [7]:
if __name__ == "__main__":
    is_hourly = False
    output_name = os.path.join(base_path + "Ameriflux/", "Jan-11-2020_Daily/")
    am = Ameriflux(daily_classified_path, skipRowsNum, split_num, lags_count, is_hourly, False, output_name)
    sites_df = Helpers.read_sites_data()
    df_daily = am.generate_site_data(sites_df)
    file_name = os.path.join(output_name, "All_Daily")
    Helpers.export_data(df_daily, file_name) 
    print(all_sites_df.head())

Initializer
Site: US-Ced
Before removing missing values:
number of rows: 1834 
number of columns: 14
After removing missing values:
number of rows: 1834 
number of columns: 14
ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED', 'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2', 'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5', 'C_BOWENS']
columns: Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
   

Site: US-Skr
Before removing missing values:
number of rows: 384 
number of columns: 14
After removing missing values:
number of rows: 384 
number of columns: 14
ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED', 'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2', 'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5', 'C_BOWENS']
columns: Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
       'G-5', 'H'

Site: US-AR1
Before removing missing values:
number of rows: 1169 
number of columns: 14
After removing missing values:
number of rows: 1169 
number of columns: 14
ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED', 'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2', 'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5', 'C_BOWENS']
columns: Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
       'G-5', '

Valid Sites are ['US-Ced', 'US-Shd', 'US-Bi2', 'US-A32', 'US-Tw2', 'US-Pon', 'US-Skr', 'US-Snd', 'US-AR2', 'US-Goo', 'US-Wlr', 'US-A74', 'US-AR1', 'US-Kon', 'US-SO2', 'US-SP2', 'US-Var', 'US-Twt']
      Timestamp start  TIMESTAMP_END Site Id  Year  Month  Day        WS  \
9744     200907230000   200907230030  US-Twt  2009      7   23  2.578392   
9748     200907230200   200907230230  US-Twt  2009      7   23  3.329627   
9750     200907230300   200907230330  US-Twt  2009      7   23  3.770757   
9751     200907230330   200907230400  US-Twt  2009      7   23  2.996385   
9752     200907230400   200907230430  US-Twt  2009      7   23  1.757107   

          WS-1      WS-2      WS-3  ...  NETRAD_SOIL         LE    LE(mm)  \
9744       NaN       NaN       NaN  ...          0.0  27.281298  0.942685   
9748  2.578392       NaN       NaN  ...          0.0  25.882064  0.894335   
9750  3.329627  2.578392       NaN  ...          0.0  23.188757  0.801270   
9751  3.770757  3.329627  2.578392  ..

<h3> Joint Data - EEflux </h3>

In [9]:
class JointData:
     def __init__(self, output_file, lags_count):
        print("Merged Initializer")
        self.output_file = output_file
        self.lags_count = lags_count
     
     def read_data(self):
        '''This function reads the data from EEflux'''
        file_name = os.path.join(base_path, "EEflux/EEflux_sites.csv")
        eeflux_df = pd.read_csv(file_name, index_col=None, header=0)
        eeflux_df["Date"] = pd.to_datetime(eeflux_df["Date"])
        eeflux_df.head()
        return eeflux_df
    
     def merge_datasets(self, df_first, df_second, col):
        '''This function will merge two data frames according to the provided columns'''
        merged_df = pd.merge(df_first, df_second, left_on=col, 
                         right_on=col, how="inner")
        columns = merged_df.columns
        return merged_df
    
     def drop_invalid_lags(self, df):
        '''This function will drop all the lags'''
        ws_list = list(Helpers.get_all_matching_columns(df, "WS-"))
        rh_list = list(Helpers.get_all_matching_columns(df, "RH-"))
        ta_list = list(Helpers.get_all_matching_columns(df, "TA-"))
        g_list = list(Helpers.get_all_matching_columns(df, "G-"))
#         h_list = list(Helpers.get_all_matching_columns(df, "H-"))
        h_corr_list = list(Helpers.get_all_matching_columns(df, "H_CORRECTED-"))
        netrad_list = list(Helpers.get_all_matching_columns(df, "NETRAD-"))
        le_list = list(Helpers.get_all_matching_columns(df, "LE(mm)-"))
        le_corr_list = list(Helpers.get_all_matching_columns(df, "LE_CORRECTED(mm)-"))

        columns_to_drop = []
        columns_to_drop.extend(ws_list)
        columns_to_drop.extend(rh_list)
        columns_to_drop.extend(ta_list)
        columns_to_drop.extend(g_list)
#         columns_to_drop.extend(h_list)
        columns_to_drop.extend(h_corr_list)
        columns_to_drop.extend(netrad_list)
        columns_to_drop.extend(le_list)
        columns_to_drop.extend(le_corr_list)
        df.drop(columns_to_drop, axis=1, inplace=True)
        
        return df
    
     def order_common_columns(self, df):
        '''This function will order the date and main columns'''
        all_columns = ["Date", "Site Id", "Year", "Month", "Day"]
        return all_columns
            
     def order_columns(self, df):
        '''This function will specify the columns required and will order the columns'''
        all_columns = self.order_common_columns(df)
        if "WS" in df.columns:
            all_columns.append("WS")
        ws_list = [col for col in df if col.startswith('WS-')]
        all_columns.extend(ws_list)
        if "RH" in df.columns:
            all_columns.append("RH")
        rh_list = [col for col in df if col.startswith('RH-')]
        all_columns.extend(rh_list)
        if "TA" in df.columns:
            all_columns.append("TA")
        ta_list = [col for col in df if col.startswith('TA-')]
        all_columns.extend(ta_list)
        if "G" in df.columns:
            all_columns.append("G")
        g_list = [col for col in df if col.startswith('G-')]
        all_columns.extend(g_list)
        if "H" in df.columns:
            all_columns.append("H")
        h_list = [col for col in df if col.startswith('H-')]
        h_corr_list = [col for col in df if col.startswith('H_')]
        all_columns.extend(h_list)
        all_columns.extend(h_corr_list)
        netrad_list = [col for col in df if col.startswith('NETRAD')]
        le_list = [col for col in df if col.startswith('LE')]
        all_columns.extend(netrad_list)
        all_columns.extend(le_list)
        all_columns.extend(['Cloud','Image Id', 'EEflux ET', 'Tier'])
        print("ordered columns", all_columns)
        df = df[all_columns]
        return df
    
     def generate_lags(self, df):
        '''This function generates the lags for the list of columns'''
        input_columns = ["RH", "TA", "G", "H_CORRECTED", "WS", "NETRAD", "LE_CORRECTED(mm)"]
        for k in range(len(input_columns)):
            col = input_columns[k]
            if col in df.columns:
                df = Helpers.generate_lags(df, col, self.lags_count)
        return df
    

     def prepare_data(self, df_daily, col_first, col_second):
        eeflux_df = self.read_data()
        df_daily["Date"] = pd.to_datetime(df_daily["Date"])
        print("daily df columns", df_daily.columns)
        group_sites_list = []
        merged_df = self.merge_datasets(df_daily, eeflux_df, ["Site Id","Date"])
        unique_sites = df_daily["Site Id"].unique()
        for i in range(len(unique_sites)):
            site_id = unique_sites[i]
            df_joint = merged_df[merged_df["Site Id"] == site_id]
            df_joint["Site Id"] = site_id
            print("Site ID", site_id)
            if len(df_joint) > 1:
                df_joint["EEflux ET"] = np.where(df_joint["Modeled ET"] == np.nan, df_joint["Mean Modeled ET"], df_joint["Modeled ET"])
                df_joint = self.drop_invalid_lags(df_joint)
                df_joint = self.generate_lags(df_joint)
                df_joint = self.order_columns(df_joint)
                df_joint = Helpers.drop_nan_columns(df_joint)
                print("columns", df_joint.columns)
                file_name = os.path.join(self.output_file, site_id + "_" + "Joint")
                Helpers.export_data(df_joint, file_name) 
                group_sites_list.append(df_joint)
            
        return Helpers.list_to_df(group_sites_list)


In [10]:
#Generate excel with all the sites Ameriflux joint with EEflux
output_name = os.path.join(base_path + "Ameriflux/", "Jan-11-2020_Joint/")
joint = JointData(output_name, lags_count)
merged_df = joint.prepare_data(df_daily, ["Site Id","Date"], ["Site Id", "Date"])
merged_df = joint.order_columns(merged_df)
file_name = os.path.join(output_name, "All_Joint")
Helpers.export_data(merged_df, file_name) 

Merged Initializer
daily df columns Index(['C_BOWENS', 'Date', 'Day', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H',
       'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3',
       'H_CORRECTED-4', 'H_CORRECTED-5', 'LE', 'LE_CORRECTED',
       'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2',
       'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5',
       'Month', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4',
       'NETRAD-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'Site Id',
       'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'WS', 'WS-1', 'WS-2',
       'WS-3', 'WS-4', 'WS-5', 'Year'],
      dtype='object')
Site ID US-Ced
ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTE

/Users/saraawad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/saraawad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/saraawad/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guid

ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED', 'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2', 'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5', 'Cloud', 'Image Id', 'EEflux ET', 'Tier']
columns Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
       'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2',
       'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1',
  

ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1', 'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED', 'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2', 'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5', 'Cloud', 'Image Id', 'EEflux ET', 'Tier']
columns Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
       'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2',
       'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1',
  

columns Index(['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3',
       'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA',
       'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4',
       'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2',
       'H_CORRECTED-3', 'H_CORRECTED-4', 'H_CORRECTED-5', 'NETRAD', 'NETRAD-1',
       'NETRAD-2', 'NETRAD-3', 'NETRAD-4', 'NETRAD-5', 'LE', 'LE_CORRECTED',
       'LE_CORRECTED(mm)', 'LE_CORRECTED(mm)-1', 'LE_CORRECTED(mm)-2',
       'LE_CORRECTED(mm)-3', 'LE_CORRECTED(mm)-4', 'LE_CORRECTED(mm)-5',
       'Cloud', 'Image Id', 'EEflux ET', 'Tier'],
      dtype='object')
Site ID US-Var
ordered columns ['Date', 'Site Id', 'Year', 'Month', 'Day', 'WS', 'WS-1', 'WS-2', 'WS-3', 'WS-4', 'WS-5', 'RH', 'RH-1', 'RH-2', 'RH-3', 'RH-4', 'RH-5', 'TA', 'TA-1', 'TA-2', 'TA-3', 'TA-4', 'TA-5', 'G', 'G-1', 'G-2', 'G-3', 'G-4', 'G-5', 'H', 'H_CORRECTED', 'H_CORRECTED-1', 'H_CORRECTED-2', 'H_CORREC

In [ ]:
class AmerifluxErrorGraph:
    def __init__(self, path):
        print("AmerifluxErrorGraph Initializer")
        self.path = path
          
    def read_data(self):
        file_name = os.path.join(base_path, self.path)
        df = pd.read_csv(file_name, index_col=None, header=0)
        df = Helpers.add_LE_conversion_rate(df, "LE")
        return df
    

    def get_error_metrics(self, y_true, y_predicted):
        r2_Score = r2_score(y_true, y_predicted)
        rmse_score = np.sqrt(mean_squared_error(y_true, y_predicted))
        mse_score = mean_squared_error(y_true, y_predicted)
        mae_score = mean_absolute_error(y_true, y_predicted)

        def mean_absolute_percentage_error(y_true, y_pred):
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        mape_score = mean_absolute_percentage_error(y_true, y_predicted)
        num = 2
        return (round(r2_Score, num), round(rmse_score, num), round(mse_score, num), 
                round(mae_score, num), round(mape_score, num))


    def generate_errors(self, df, first_col, second_col):
        df = df.replace(to_replace = np.nan, value =0) 
        errors = self.get_error_metrics(df[first_col], df[second_col])
        return errors
    
    def plot_et(self, df, site_id, first_column, first_b_column, second_column, path_to_save):
        fig, ax = plt.subplots(figsize=(40, 17))
        fig.subplots_adjust(bottom=0.15, left=0.2)
        plt.subplot(121)
        title1 = site_id + ": Comparison before correction between " + first_column + " and " + second_column
        self.plot_sub(df, first_column, second_column, title1)
        
        plt.subplot(122)
        title2 = site_id + ": Comparison after correction between " + first_b_column + " and " + second_column
        self.plot_sub(df, first_b_column, second_column, title2)

        plt.xticks(rotation=90)
#         ax.legend()
#         plt.show()
        full_path = os.path.join(base_path, path_to_save)
        plt.savefig(full_path + site_id + "_et.png")
        plt.close(fig)

    def plot_sub(self, df, first_column, second_column, title):
        ax = sns.regplot(x=first_column, y=second_column, data=df, fit_reg=False)
        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
            np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
            ]

        # now plot both limits against eachother
        ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(lims)
        ax.set_ylim(lims)
       
        plt.title(title)
        
        
    def prepare_data(self, df):
        unique_sites = df["Site Id"].unique()
        unique_sites

        r2_list, rmse_list, mse_list, mae_list, mape_list = [], [], [], [], []
        first_column = "LE(mm)"
        first_b_column = "LE_CORRECTED(mm)"

        second_column = "EEflux ET"
        for i in range(len(unique_sites)):
            site_id = unique_sites[i]
            df_site = df[df["Site Id"] == site_id]
            (r2, rmse, mse, mae, mape) = self.generate_errors(df_site, first_column, second_column)
            r2_list.append(r2)
            rmse_list.append(rmse)
            mse_list.append(mse)
            mae_list.append(mae)
            mape_list.append(mape)
            df_site['Date'] = pd.to_datetime(df_site["Date"])
            df_site.sort_values(by="Date", inplace=True, ascending=True)
            #Plot the output feature
            if len(df_site) > 0:
                self.plot_et(df_site, site_id, first_column, first_b_column, second_column, "Ameriflux/Generated/v1/Graphs/")

        df_errors = pd.DataFrame({"Site Id": unique_sites,
                                  "True Value": first_column,
                                  "Predicted Value": second_column,
                                  "R2": r2_list,
                                  "RMSE": rmse_list, 
                                  "MSE": rmse_list,
                                  "MAE": mae_list, 
                                  "MAPE": mape_list})
        print(df_errors)
        errors_path = "Ameriflux/Generated/v1/Errors_ET"
        Helpers.export_data(df_errors, errors_path)



full_path = os.path.join("/Users/saraawad/Desktop/All_Joint.csv")
am_eg = AmerifluxErrorGraph(full_path)
df = am_eg.read_data()
df.head()
am_eg.prepare_data(df)

AmerifluxErrorGraph Initializer
